In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from keras.layers import TextVectorization
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, Activation, Embedding, Bidirectional
from keras.optimizers import Adam

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.10 from "c:\Users\yann.MSI\anaconda3\envs\data_science\python.exe"
  * The NumPy version is: "1.23.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'


In [37]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


Load Dataset

In [23]:
train_dataset, test_dataset = tfds.load("imdb_reviews", as_supervised=True, split=["train", "test"])

Preprocess Dataset

In [24]:
text_vec = TextVectorization(
        max_tokens=20_000,
        output_mode="int",
        output_sequence_length=80
    )

def preprocess_model() -> Sequential:
    model = Sequential()
    model.add(Input(shape=(1,), dtype=tf.string))
    model.add(text_vec)
    return model

def one_hot(y:tf.Tensor) -> tf.Tensor:
    y = tf.one_hot(tf.cast(y, tf.int32), depth=2)
    y = tf.cast(y, tf.float32)
    return y

preprocessing_model = preprocess_model()
words = train_dataset.map(lambda x,y: x)
text_vec.adapt(words)
train_dataset = train_dataset.map(lambda x,y: (tf.expand_dims(tf.cast(preprocessing_model(x, training=False), tf.int32), axis=-1), tf.expand_dims(y, axis=-1)))
test_dataset = test_dataset.map(lambda x,y: (preprocessing_model(x, training=False), one_hot(y)))


In [25]:
print(train_dataset)
for text, label in train_dataset.take(2):
    print(text)
    print(label)

<_MapDataset element_spec=(TensorSpec(shape=(None, 1), dtype=tf.int32, name=None), TensorSpec(shape=(1,), dtype=tf.int64, name=None))>
tf.Tensor(
[[   11]
 [   14]
 [   34]
 [  412]
 [  384]
 [   18]
 [   90]
 [   28]
 [10690]
 [    8]
 [   33]
 [ 1322]
 [ 3560]
 [   42]
 [  487]
 [11832]
 [  191]
 [   24]
 [   85]
 [  152]
 [   19]
 [   11]
 [  217]
 [  316]
 [   28]
 [   65]
 [  240]
 [  214]
 [    8]
 [  489]
 [   54]
 [   65]
 [   85]
 [  112]
 [   96]
 [   22]
 [ 5596]
 [   11]
 [   93]
 [  642]
 [  743]
 [   11]
 [   18]
 [    7]
 [   34]
 [  394]
 [ 9522]
 [  170]
 [ 2464]
 [  408]
 [    2]
 [   88]
 [ 1216]
 [  137]
 [   66]
 [  144]
 [   51]
 [    2]
 [    1]
 [ 7558]
 [   66]
 [  245]
 [   65]
 [ 2870]
 [   16]
 [18297]
 [ 2860]
 [    1]
 [    1]
 [ 1426]
 [ 5050]
 [    3]
 [   40]
 [    1]
 [ 1579]
 [   17]
 [ 3560]
 [   14]
 [  158]
 [   19]], shape=(80, 1), dtype=int32)
tf.Tensor([0], shape=(1,), dtype=int64)
tf.Tensor(
[[  10]
 [  26]
 [  75]
 [ 617]
 [   6]
 [ 776]
 [235

Create Model

In [26]:
def create_model(
    input_shape: tuple[int, int],
    num_classes: int,
    embedding_dim: int
)->Model:
    
    input_text = Input(shape=input_shape)
    x = Embedding(
        input_dim= 20_000, #vocab size
        output_dim=embedding_dim,
        input_length=80 #sequence length
    )(input_text)
    x = LSTM(units=80, return_sequences=False)(x)
    x = Dense(units=80)(x)
    x = Activation("relu")(x)
    x = Dense(units=1)(x)
    output = Activation("sigmoid")(x)
    model = Model(inputs=[input_text], outputs=[output])
    
    opt = Adam(learning_rate=1e-4)
    model.compile(
        loss="binary_crossentropy",
        optimizer=opt,
        metrics=["accuracy"]
    )
    model.summary()
    return model

In [ ]:

input_shape = (80) #sequence length
num_classes = 2
epochs = 10
batch_size = 512
embedding_dim = 50
model = create_model(input_shape=input_shape, num_classes=num_classes, embedding_dim=embedding_dim)
model.fit(x=train_dataset, verbose=1, batch_size=batch_size, epochs=epochs)
